# 🚀 Step 1: Align & Resample All Images


Since orthomosaics might have different spatial extents and resolutions, we first align them using a reference CRS and resample them to a uniform size.

In [1]:
import os
import numpy as np
import rasterio
from rasterio.warp import reproject, calculate_default_transform, Resampling
from PIL import Image
import imageio

def align_and_resample(input_path, ref_crs, ref_transform, ref_shape, output_path):
    """
    Aligns and resamples an image to match a reference CRS and resolution.
    
    Args:
        input_path (str): Path to the input TIFF file.
        ref_crs (CRS): Reference CRS for alignment.
        ref_transform (Affine): Reference transformation matrix.
        ref_shape (tuple): Reference shape (height, width).
        output_path (str): Path to save the aligned image.
    
    Returns:
        str: Path of the aligned and resampled image.
    """
    with rasterio.open(input_path) as src:
        transform, width, height = calculate_default_transform(
            src.crs, ref_crs, src.width, src.height, *src.bounds
        )

        profile = src.profile.copy()
        profile.update(transform=transform, width=width, height=height, crs=ref_crs)

        aligned_image = np.zeros((src.count, height, width), dtype=src.dtypes[0])

        for i in range(src.count):
            reproject(
                source=src.read(i + 1),
                destination=aligned_image[i],
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=ref_crs,
                resampling=Resampling.bilinear,
            )

        with rasterio.open(output_path, "w", **profile) as dst:
            for i in range(src.count):
                dst.write(aligned_image[i], i + 1)

    print(f"✅ Aligned & Resampled: {output_path}")
    return output_path


# 🚀 Step 2: Process All Orthomosaics
We need to:

Find all images in the given folders.
Sort them properly (e.g., by date, name, or field).
Align them using a reference (first image as baseline).
Convert to PNG format for animation.

In [2]:
def process_orthomosaics(image_folders, output_folder):
    """
    Process all orthomosaics by aligning and converting them to PNG for animation.
    
    Args:
        image_folders (list): List of folders containing TIFF images (NDVI, RGB).
        output_folder (str): Folder to save processed images.
    
    Returns:
        list: List of paths to processed PNG images.
    """
    os.makedirs(output_folder, exist_ok=True)
    
    tiff_files = []
    for folder in image_folders:
        for file in sorted(os.listdir(folder)):  # Sort for correct order
            if file.endswith(".tif"):
                tiff_files.append(os.path.join(folder, file))

    if not tiff_files:
        print("❌ No TIFF files found!")
        return []

    # Use the first image as the reference
    with rasterio.open(tiff_files[0]) as ref_src:
        ref_crs = ref_src.crs
        ref_transform = ref_src.transform
        ref_shape = (ref_src.height, ref_src.width)

    processed_images = []

    for idx, tif_path in enumerate(tiff_files):
        aligned_path = os.path.join(output_folder, f"aligned_{idx}.tif")
        png_path = aligned_path.replace(".tif", ".png")

        # Align and resample
        aligned_tif = align_and_resample(tif_path, ref_crs, ref_transform, ref_shape, aligned_path)

        # Convert to PNG
        with rasterio.open(aligned_tif) as src:
            img_data = np.dstack([src.read(i + 1) for i in range(min(3, src.count))])  # Stack up to 3 bands
        
            print("Image shape before conversion:", img_data.shape)  # Debugging
        
            # Ensure valid shape (at least H, W)
            if img_data.ndim < 3 or img_data.shape[0] < 1 or img_data.shape[1] < 1:
                print("⚠️ Skipping empty or invalid image!")
                continue
        
            # Ensure the correct data type
            img_data = img_data.astype(np.uint8)  
        
            # Convert single-band grayscale to RGB
            if img_data.shape[2] == 1:
                img_data = np.repeat(img_data, 3, axis=2)  # Convert (H, W, 1) → (H, W, 3)
        
            # Ensure it's 3 channels (RGB)
            if img_data.shape[2] > 3:
                img_data = img_data[:, :, :3]  # Trim excess channels
        
            # Convert to PIL image
            img = Image.fromarray(img_data)
        
            # Save as PNG
            img.save(png_path)
        
        processed_images.append(png_path)

    return processed_images


# 🚀 Step 3: Animate the Processed Images
Now, let's animate the aligned images into a GIF or MP4.

## Incomplete: works align images but rgb is two tone¶

In [70]:
import rasterio
import numpy as np
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.merge import merge
import imageio.v2 as imageio
import os

def create_animation(image_list, output_path, fps=2, quality="high", format="both"):
    """
    Create an animation (GIF and/or MP4) from a list of georeferenced images.

    Args:
        image_list (list): List of file paths for georeferenced images.
        output_path (str): Output file path (without extension).
        fps (int): Frames per second.
        quality (str): Output quality - "low", "medium", "high", or "original".
        format (str): Output format - "gif", "mp4", or "both".
    """

    if not image_list:
        print("⚠️ No images found! Skipping animation.")
        return

    if format not in ["gif", "mp4", "both"]:
        print("⚠️ Invalid format! Defaulting to 'both'.")
        format = "both"

    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    # Quality settings
    quality_settings = {
        "low": {"scale_factor": 0.3},
        "medium": {"scale_factor": 0.6},
        "high": {"scale_factor": 1.0},
        "original": {"scale_factor": 1.0},
    }

    if quality not in quality_settings:
        print("⚠️ Invalid quality setting! Defaulting to 'high'.")
        quality = "high"

    scale_factor = quality_settings[quality]["scale_factor"]

    print(f"\n📢 Creating animation with {len(image_list)} frames...")

    # Determine the common extent and resolution
    with rasterio.open(image_list[0]) as src:
        dst_crs = src.crs
        dst_bounds = src.bounds

    for img_path in image_list[1:]:
        with rasterio.open(img_path) as src:
            dst_bounds = (
                min(dst_bounds[0], src.bounds[0]),
                min(dst_bounds[1], src.bounds[1]),
                max(dst_bounds[2], src.bounds[2]),
                max(dst_bounds[3], src.bounds[3]),
            )

    # Calculate the transform for the output
    with rasterio.open(image_list[0]) as src:
        dst_transform, dst_width, dst_height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *dst_bounds
        )

    # Apply scaling factor
    dst_width = int(dst_width * scale_factor)
    dst_height = int(dst_height * scale_factor)
    dst_transform *= src.transform.scale(
        (src.width / dst_width),
        (src.height / dst_height)
    )

    processed_frames = []
    for i, img_path in enumerate(image_list):
        print(f"🔄 Processing image {i + 1} of {len(image_list)}...")
        
        with rasterio.open(img_path) as src:
            # Reproject and rescale the image
            reproject_kwargs = dict(
                source=rasterio.band(src, 1),
                destination=np.zeros((dst_height, dst_width), dtype=np.uint8),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=dst_transform,
                dst_crs=dst_crs,
                resampling=Resampling.bilinear
            )
            
            reprojected_img, _ = reproject(**reproject_kwargs)
            processed_frames.append(reprojected_img)

    # Save GIF if requested
    if format in ["gif", "both"]:
        gif_path = f"{output_path}.gif"
        imageio.mimsave(gif_path, processed_frames, duration=1/fps)
        print(f"✅ GIF animation saved: {gif_path}")

    # Save MP4 if requested
    if format in ["mp4", "both"]:
        mp4_path = f"{output_path}.mp4"
        try:
            imageio.mimsave(mp4_path, processed_frames, fps=fps, codec='libx264', bitrate='5000k')
            print(f"✅ MP4 animation saved: {mp4_path}")
        except Exception as e:
            print(f"❌ Failed to generate MP4: {str(e)}")
            print("   This might be due to missing FFMPEG. Please ensure FFMPEG is installed and accessible.")

# Example usage:
# create_animation(image_list, "output/animation", fps=2, quality="high", format="both")


## Works with gif speed and rgb animation colored but speed control is not working

In [51]:
import rasterio
import numpy as np
from rasterio.warp import calculate_default_transform, reproject, Resampling
import imageio.v2 as imageio
import os

def create_animation(image_list, output_path, fps=2, quality="high", format="both", duration=0.5):
    """
    Create an animation (GIF and/or MP4) from a list of georeferenced images.

    Args:
        image_list (list): List of file paths for georeferenced images.
        output_path (str): Output file path (without extension).
        fps (int): Frames per second (for MP4).
        quality (str): Output quality - "low", "medium", "high", or "original".
        format (str): Output format - "gif", "mp4", or "both".
        duration (float): Duration of each frame in seconds (for GIF).
    """

    if not image_list:
        print("⚠️ No images found! Skipping animation.")
        return

    if format not in ["gif", "mp4", "both"]:
        print("⚠️ Invalid format! Defaulting to 'both'.")
        format = "both"

    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    # Quality settings
    quality_settings = {
        "low": {"scale_factor": 0.3},
        "medium": {"scale_factor": 0.6},
        "high": {"scale_factor": 1.0},
        "original": {"scale_factor": 1.0},
    }

    if quality not in quality_settings:
        print("⚠️ Invalid quality setting! Defaulting to 'high'.")
        quality = "high"

    scale_factor = quality_settings[quality]["scale_factor"]

    print(f"\n📢 Creating animation with {len(image_list)} frames...")

    # Determine the common extent and resolution
    with rasterio.open(image_list[0]) as src:
        dst_crs = src.crs
        dst_bounds = src.bounds

    for img_path in image_list[1:]:
        with rasterio.open(img_path) as src:
            dst_bounds = (
                min(dst_bounds[0], src.bounds[0]),
                min(dst_bounds[1], src.bounds[1]),
                max(dst_bounds[2], src.bounds[2]),
                max(dst_bounds[3], src.bounds[3]),
            )

    # Calculate the transform for the output
    with rasterio.open(image_list[0]) as src:
        dst_transform, dst_width, dst_height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *dst_bounds
        )

    # Apply scaling factor
    dst_width = int(dst_width * scale_factor)
    dst_height = int(dst_height * scale_factor)
    dst_transform = rasterio.transform.from_bounds(*dst_bounds, width=dst_width, height=dst_height)

    processed_frames = []
    for i, img_path in enumerate(image_list):
        print(f"🔄 Processing image {i + 1} of {len(image_list)}...")
        
        with rasterio.open(img_path) as src:
            # Read all bands
            img = src.read()
            
            # Handle single-band images (e.g., NDVI)
            if img.shape[0] == 1:
                # Normalize to 0-255 range for visualization
                img = img.squeeze()
                img = ((img - img.min()) / (img.max() - img.min()) * 255).astype(np.uint8)
                img = np.stack([img, img, img])  # Create RGB image
            
            # Reproject and rescale the image
            reproject_kwargs = dict(
                source=img,
                destination=np.zeros((img.shape[0], dst_height, dst_width), dtype=np.uint8),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=dst_transform,
                dst_crs=dst_crs,
                resampling=Resampling.bilinear
            )
            
            reprojected_img, _ = reproject(**reproject_kwargs)
            
            # Transpose the image to have channels last (height, width, channels)
            reprojected_img = np.transpose(reprojected_img, (1, 2, 0))
            
            # Clip values to 0-255 range and convert to uint8
            reprojected_img = np.clip(reprojected_img, 0, 255).astype(np.uint8)
            
            processed_frames.append(reprojected_img)

    # Save GIF if requested
    if format in ["gif", "both"]:
        gif_path = f"{output_path}.gif"
        imageio.mimsave(gif_path, processed_frames, duration=duration)
        print(f"✅ GIF animation saved: {gif_path}")

    # Save MP4 if requested
    if format in ["mp4", "both"]:
        mp4_path = f"{output_path}.mp4"
        try:
            imageio.mimsave(mp4_path, processed_frames, fps=fps, codec='libx264', bitrate='5000k')
            print(f"✅ MP4 animation saved: {mp4_path}")
        except Exception as e:
            print(f"❌ Failed to generate MP4: {str(e)}")
            print("   This might be due to missing FFMPEG. Please ensure FFMPEG is installed and accessible.")

# Example usage:
# create_animation(image_list, "output/animation", fps=2, quality="high", format="both", duration=0.5)


# NDVI is still not working and gif speed control, MP4 not working

In [65]:
import rasterio
import numpy as np
from rasterio.warp import calculate_default_transform, reproject, Resampling
import imageio.v2 as imageio
import os

def create_animation(image_list, output_path, fps=2, quality="high", format="both", duration=1.0):
    """
    Create an animation (GIF and/or MP4) from a list of georeferenced images.

    Args:
        image_list (list): List of file paths for georeferenced images.
        output_path (str): Output file path (without extension).
        fps (int): Frames per second (for MP4).
        quality (str): Output quality - "low", "medium", "high", or "original".
        format (str): Output format - "gif", "mp4", or "both".
        duration (float): Duration of each frame in seconds (for GIF).
    """

    if not image_list:
        print("⚠️ No images found! Skipping animation.")
        return

    if format not in ["gif", "mp4", "both"]:
        print("⚠️ Invalid format! Defaulting to 'both'.")
        format = "both"

    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    # Quality settings
    quality_settings = {
        "low": {"scale_factor": 0.3},
        "medium": {"scale_factor": 0.6},
        "high": {"scale_factor": 1.0},
        "original": {"scale_factor": 1.0},
    }

    if quality not in quality_settings:
        print("⚠️ Invalid quality setting! Defaulting to 'high'.")
        quality = "high"

    scale_factor = quality_settings[quality]["scale_factor"]

    print(f"\n📢 Creating animation with {len(image_list)} frames...")

    # Determine the common extent and resolution
    with rasterio.open(image_list[0]) as src:
        dst_crs = src.crs
        dst_bounds = src.bounds
        num_bands = src.count

    for img_path in image_list[1:]:
        with rasterio.open(img_path) as src:
            dst_bounds = (
                min(dst_bounds[0], src.bounds[0]),
                min(dst_bounds[1], src.bounds[1]),
                max(dst_bounds[2], src.bounds[2]),
                max(dst_bounds[3], src.bounds[3]),
            )

    # Calculate the transform for the output
    with rasterio.open(image_list[0]) as src:
        dst_transform, dst_width, dst_height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *dst_bounds
        )

    # Apply scaling factor
    dst_width = int(dst_width * scale_factor)
    dst_height = int(dst_height * scale_factor)
    dst_transform = rasterio.transform.from_bounds(*dst_bounds, width=dst_width, height=dst_height)

    processed_frames = []
    for i, img_path in enumerate(image_list):
        print(f"🔄 Processing image {i + 1} of {len(image_list)}...")
        
        with rasterio.open(img_path) as src:
            # Read all bands
            img = src.read()
            
            # Reproject and rescale the image
            reproject_kwargs = dict(
                source=img,
                destination=np.zeros((num_bands, dst_height, dst_width), dtype=np.float32),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=dst_transform,
                dst_crs=dst_crs,
                resampling=Resampling.bilinear
            )
            
            reprojected_img, _ = reproject(**reproject_kwargs)
            
            # Normalize each band to 0-255 range
            normalized_img = np.zeros_like(reprojected_img, dtype=np.uint8)
            for band in range(num_bands):
                band_data = reprojected_img[band]
                normalized_img[band] = np.interp(band_data, (band_data.min(), band_data.max()), (0, 255))
            
            # Ensure we have at least 3 bands for RGB representation
            if num_bands < 3:
                normalized_img = np.repeat(normalized_img, 3, axis=0)[:3]
            elif num_bands > 3:
                normalized_img = normalized_img[:3]  # Use first 3 bands for RGB
            
            # Transpose the image to have channels last (height, width, channels)
            normalized_img = np.transpose(normalized_img, (1, 2, 0))
            
            processed_frames.append(normalized_img)

    # Save GIF if requested
    if format in ["gif", "both"]:
        gif_path = f"{output_path}.gif"
        imageio.mimsave(gif_path, processed_frames, duration=duration)
        print(f"✅ GIF animation saved: {gif_path}")

    # Save MP4 if requested
    if format in ["mp4", "both"]:
        mp4_path = f"{output_path}.mp4"
        try:
            imageio.mimsave(mp4_path, processed_frames, fps=fps, codec='libx264', bitrate='5000k')
            print(f"✅ MP4 animation saved: {mp4_path}")
        except Exception as e:
            print(f"❌ Failed to generate MP4: {str(e)}")
            print("   This might be due to missing FFMPEG. Please ensure FFMPEG is installed and accessible.")

# Example usage:
# create_animation(image_list, "output/animation", fps=2, quality="high", format="both", duration=1.0)


# 🚀 Step 4: Run Everything
Now, we just need to call everything in sequence.

In [66]:
import glob
# Function to find the tif files in a given folder¶
def find_files_in_folder(folder_path, extension=None, recursive=False):
    matched_files = []
    # Determine the search pattern based on whether an extension is provided and recursion is enabled
    if extension:
        if recursive:
            search_pattern = os.path.join(folder_path, f"**/*.{extension}")
        else:
            search_pattern = os.path.join(folder_path, f"*.{extension}")
    else:
        # No extension specified, handle both recursive and non-recursive cases
        if recursive:
            search_pattern = os.path.join(folder_path, "**/*")
        else:
            search_pattern = os.path.join(folder_path, "*")
    # Use glob to find matching files in the specified directory and subdirectories if recursive
    matched_files.extend(glob.glob(search_pattern, recursive=recursive))
    # If no files are found, return a list with an empty string
    if not matched_files:
        matched_files = [""]
    return matched_files

In [67]:
import os
os.environ["IMAGEIO_FFMPEG_EXE"] = "/usr/bin/ffmpeg"

In [68]:
# Folders containing NDVI and RGB orthomosaics
ndvi_folder = r"D:\PhenoCrop\3_orthomosaics_rgb_ndvi\PRO_BAR_VOLL\animation\NDVI"
rgb_folder = r"D:\PhenoCrop\3_orthomosaics_rgb_ndvi\PRO_BAR_VOLL\animation\RGB"
output_folder = r"D:\PhenoCrop\3_orthomosaics_rgb_ndvi\PRO_BAR_VOLL\animation"

ndvi_images = find_files_in_folder(ndvi_folder, "tif")
rgb_images = find_files_in_folder(rgb_folder, "tif")

output_folder = r"D:\PhenoCrop\3_orthomosaics_rgb_ndvi\PRO_BAR_VOLL\animation"
images = find_files_in_folder(output_folder, "tif")

create_animation(rgb_images, os.path.join(output_folder, "anim_rgb"), fps=1, quality="high", format="gif", duration=2)   # 1 second per frame
create_animation(ndvi_images, os.path.join(output_folder, "anim_ndvi"), fps=1, quality="high", format="gif", duration=2)  # 1 second per frame


📢 Creating animation with 13 frames...
🔄 Processing image 1 of 13...
🔄 Processing image 2 of 13...
🔄 Processing image 3 of 13...
🔄 Processing image 4 of 13...
🔄 Processing image 5 of 13...
🔄 Processing image 6 of 13...
🔄 Processing image 7 of 13...
🔄 Processing image 8 of 13...
🔄 Processing image 9 of 13...
🔄 Processing image 10 of 13...
🔄 Processing image 11 of 13...
🔄 Processing image 12 of 13...
🔄 Processing image 13 of 13...
✅ GIF animation saved: D:\PhenoCrop\3_orthomosaics_rgb_ndvi\PRO_BAR_VOLL\animation\anim_rgb.gif

📢 Creating animation with 15 frames...
🔄 Processing image 1 of 15...
🔄 Processing image 2 of 15...
🔄 Processing image 3 of 15...
🔄 Processing image 4 of 15...
🔄 Processing image 5 of 15...
🔄 Processing image 6 of 15...
🔄 Processing image 7 of 15...
🔄 Processing image 8 of 15...
🔄 Processing image 9 of 15...
🔄 Processing image 10 of 15...
🔄 Processing image 11 of 15...
🔄 Processing image 12 of 15...
🔄 Processing image 13 of 15...
🔄 Processing image 14 of 15...
🔄 Pro

In [11]:
os.path.join(output_folder, "anim")

'D:\\PhenoCrop\\3_orthomosaics_rgb_ndvi\\PRO_BAR_VOLL\\animation\\anim'

In [15]:
# Step 1: Process images
processed_images = process_orthomosaics([ndvi_folder, rgb_folder], output_folder)

# Step 2: Animate the processed images
if processed_images:
    create_animation(processed_images, "ndvi_animation.gif", fps=5, output_mp4="ndvi_animation.mp4")


✅ GIF animation saved: ndvi_animation.gif


TypeError: TiffWriter.write() got an unexpected keyword argument 'fps'